## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Import the dependencies.
import pandas as pd
import requests
import gmaps
import numpy as np
import os

# Import the API key.
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Meulaboh,ID,4.1363,96.1285,83.48,76,70,3.89,broken clouds
1,1,Khatanga,RU,71.9667,102.5000,-6.86,96,100,6.67,overcast clouds
2,2,Kodiak,US,57.7900,-152.4072,26.11,74,100,19.57,light snow
3,3,Punta Arenas,CL,-53.1500,-70.9167,59.11,44,100,5.99,overcast clouds
4,4,Namibe,AO,-15.1961,12.1522,70.86,83,100,6.26,overcast clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Meulaboh,ID,4.1363,96.1285,83.48,76,70,3.89,broken clouds
4,4,Namibe,AO,-15.1961,12.1522,70.86,83,100,6.26,overcast clouds
5,5,Saint-Joseph,RE,-21.3667,55.6167,79.11,71,0,7.00,clear sky
7,7,Hilo,US,19.7297,-155.0900,74.25,86,4,3.00,clear sky
8,8,Bluff,NZ,-46.6000,168.3333,67.80,63,91,8.05,overcast clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                254
City                   254
Country                249
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Meulaboh,ID,83.48,4.1363,96.1285,broken clouds,
4,Namibe,AO,70.86,-15.1961,12.1522,overcast clouds,
5,Saint-Joseph,RE,79.11,-21.3667,55.6167,clear sky,
7,Hilo,US,74.25,19.7297,-155.0900,clear sky,
8,Bluff,NZ,67.80,-46.6000,168.3333,overcast clouds,


In [24]:
hotel_df


,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Meulaboh,ID,83.48,4.1363,96.1285,broken clouds,
4,Namibe,AO,70.86,-15.1961,12.1522,overcast clouds,
5,Saint-Joseph,RE,79.11,-21.3667,55.6167,clear sky,
7,Hilo,US,74.25,19.7297,-155.0900,clear sky,
8,Bluff,NZ,67.80,-46.6000,168.3333,overcast clouds,
...,...,...,...,...,...,...,...
677,Fare,PF,81.70,-16.7000,-151.0167,clear sky,
679,Kasane,BW,69.89,-17.8167,25.1500,overcast clouds,
681,Kita,ML,73.78,13.0349,-9.4895,few clouds,
683,Manyana,BW,68.38,-23.4000,21.7167,few clouds,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
        
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
 
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
              

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    print("-------")
    
 # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")             

-------
-------
-------
-------
-------
-------
-------
-------
Hotel not found... skipping.
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
Hotel not found... skipping.
-------
-------
-------
-------
-------
-------
-------
Hotel not found... skipping.
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
Hotel not found... skipping.
-------
-------
-------
-------
-------
-------
Hotel not found... skipping.
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------
-------

In [28]:
# replace any empty strings in the Hotel Name column with np.nan
hotel_list= hotel_df.replace(r'^\s*$', np.nan, regex=True)

In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_list.dropna()
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Meulaboh,ID,83.48,4.1363,96.1285,broken clouds,Kost Pria
4,Namibe,AO,70.86,-15.1961,12.1522,overcast clouds,Hotel Chik Chik Namibe
5,Saint-Joseph,RE,79.11,-21.3667,55.6167,clear sky,"""Plantation Bed and Breakfast"
7,Hilo,US,74.25,19.7297,-155.0900,clear sky,Hilo Hawaiian Hotel
8,Bluff,NZ,67.80,-46.6000,168.3333,overcast clouds,Bluff Homestead - Guesthouse & Campervan Park
...,...,...,...,...,...,...,...
674,Ormara,PK,67.91,25.2088,64.6357,clear sky,Crystal Beach and Camping Resort Ormara
677,Fare,PF,81.70,-16.7000,-151.0167,clear sky,Maitai Lapita Village Huahine
679,Kasane,BW,69.89,-17.8167,25.1500,overcast clouds,The Old House
681,Kita,ML,73.78,13.0349,-9.4895,few clouds,Hôtel Chat Rouge


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation_database.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
cur_desc = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))